In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# import pickle
# output = open('pipeline.pkl', 'wb')
# pickle.dump(pipeline, output)
# output.close()

#1. WeSpeak Experiment Model

In [ ]:
!pip install git+https://github.com/wenet-e2e/wespeaker.git

In [12]:
from google.colab import files
files.upload()

Saving speech_model_embedding.bin to speech_model_embedding.bin


In [8]:
import wespeaker
import torch

In [91]:
model = wespeaker.load_model('english')
# model.set_gpu(0)
with torch.no_grad():
  diar_result = model.diarize('/content/Call-3-Example')

100%|██████████| 4/4 [02:36<00:00, 39.18s/it]


In [92]:
for _, speech_start, speech_end, speaker in diar_result:
    print(f"{speech_start:4.5f} {speech_end:4.5f} SPEAKER_0{speaker}")

0.61000 4.06000 SPEAKER_00
4.19400 5.43400 SPEAKER_00
6.69000 7.90000 SPEAKER_00
8.03400 10.01400 SPEAKER_01
10.11400 11.83400 SPEAKER_01
12.09800 14.00800 SPEAKER_01
14.97800 15.54800 SPEAKER_00
15.65000 17.02000 SPEAKER_00
17.25000 21.27000 SPEAKER_00
21.60200 23.32200 SPEAKER_00
23.42600 25.65600 SPEAKER_00
25.95400 29.27400 SPEAKER_00
29.53800 30.58800 SPEAKER_00
31.10600 31.76600 SPEAKER_00
32.22600 32.88600 SPEAKER_01
33.09000 34.01000 SPEAKER_01
34.24200 35.70200 SPEAKER_01
36.25800 37.68800 SPEAKER_00
38.11400 42.52400 SPEAKER_00
42.62600 44.50600 SPEAKER_00
44.73800 46.67800 SPEAKER_00
46.97800 50.45800 SPEAKER_00
50.85000 52.19000 SPEAKER_01
52.32200 56.86200 SPEAKER_00
56.99400 59.25400 SPEAKER_00
59.87400 60.98400 SPEAKER_01
61.41000 63.13000 SPEAKER_01
63.29800 65.84800 SPEAKER_01
66.65800 67.47800 SPEAKER_00
67.61800 68.75800 SPEAKER_00
68.99400 73.94400 SPEAKER_00
74.05000 75.92500 SPEAKER_01
75.92500 77.05000 SPEAKER_00
77.25000 80.60000 SPEAKER_00
80.73800 81.36800 SPE

**Even having GPU - It still taking too long**

# 2. Whisper Experiment Model

In [17]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

In [79]:
!pip install pydub

In [80]:
model_name = "small.en" # or medium.en

In [81]:
import torch
import wave
import whisper
import datetime
import subprocess
import contextlib
import numpy as np
import pyannote.audio

from pydub import AudioSegment
from pyannote.audio import Audio
from pyannote.core import Segment
from sklearn.cluster import AgglomerativeClustering, KMeans
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [21]:
PATH = "/content/Call-3-Example"

In [22]:
model = whisper.load_model(model_name)

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 95.2MiB/s]


In [25]:
diarization = model.transcribe(PATH)

In [26]:
print(diarization)

{'text': " Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today? Hi Anna, it's Bob. I'm wondering about the cost to send a £50 box of tea from Shanghai to London. Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded. We may contact you in the future to offer further products and services. You always have the option to withdraw from receiving this contact from us. Is that alright with you? Erm, sure, no problem now about the tea. Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box from Shanghai to London would depend on a few factors, such as the shipping method, delivery speed and any additional services you might require. For an accurate quote, could you please provide me with more details? Are you looking for standard shipping or express? Do you have any specific preferences? Erm, let's go with the standard shipping. It's just regular tea, nothi

In [27]:
conversation_convert = diarization.get("text", "")

In [28]:
conversation_convert

" Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today? Hi Anna, it's Bob. I'm wondering about the cost to send a £50 box of tea from Shanghai to London. Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded. We may contact you in the future to offer further products and services. You always have the option to withdraw from receiving this contact from us. Is that alright with you? Erm, sure, no problem now about the tea. Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box from Shanghai to London would depend on a few factors, such as the shipping method, delivery speed and any additional services you might require. For an accurate quote, could you please provide me with more details? Are you looking for standard shipping or express? Do you have any specific preferences? Erm, let's go with the standard shipping. It's just regular tea, nothing fancy.

In [30]:
embedding_model = PretrainedSpeakerEmbedding("/content/speech_model_embedding.bin")

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint speech_model_embedding.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [82]:
audio = Audio()

def segment_embedding(segment, embedding_model ,PATH):
  audio_segment = AudioSegment.from_file(PATH)
  start = segment["start"]
  end = segment["end"]
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(PATH, clip)
  audio_record = audio_segment[start:end]
  mono_waveform = torch.mean(waveform, dim=0, keepdim=True)
  return audio_record, embedding_model(mono_waveform)

In [97]:
def speech_discriminate(segments, embedding_model, num_speakers, PATH):

  embeddings = np.zeros(shape=(len(segments), 512))
  audio_list = []
  dialogue_details = []
  transcriptions_by_speaker = {}
  for i, segment in enumerate(segments):
    audio, embeddings[i] = segment_embedding(segment, embedding_model, PATH)
    audio_list.append(audio)
  embeddings = np.nan_to_num(embeddings)
  clustering = KMeans(num_speakers).fit(embeddings)
  labels = clustering.labels_
  for i in range(len(segments)):
    speaker_id = segments[i]["speaker"] = "SPEAKER_" + '0' + str(labels[i])
    voice_start = segments[i].get("start", "")
    voice_end = segments[i].get("end", "")
    voice_text = segments[i].get("text", "")
    if speaker_id not in transcriptions_by_speaker:
      transcriptions_by_speaker[speaker_id] = {"transcription": "", "segments": []}
    transcriptions_by_speaker[speaker_id]["transcription"] += " " + voice_text
    transcriptions_by_speaker[speaker_id]["segments"].append(audio_list[i])

    dialogue_detail = {"speaker": speaker_id, "transcription": voice_text}
    dialogue_details.append(dialogue_detail)
    print(f"speaker: {speaker_id} - start: {voice_start} - end: {voice_end} - text: {voice_text}" )
  return transcriptions_by_speaker, dialogue_details


In [98]:
transcriptions_by_speaker, dialogue_details = speech_discriminate(diarization.get("segments", ""), embedding_model, 2, PATH)

speaker: SPEAKER_01 - start: 0.0 - end: 6.0 - text:  Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today?
speaker: SPEAKER_00 - start: 6.0 - end: 14.0 - text:  Hi Anna, it's Bob. I'm wondering about the cost to send a £50 box of tea from Shanghai to London.
speaker: SPEAKER_01 - start: 14.0 - end: 21.0 - text:  Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded.
speaker: SPEAKER_01 - start: 21.0 - end: 25.0 - text:  We may contact you in the future to offer further products and services.
speaker: SPEAKER_01 - start: 26.0 - end: 31.0 - text:  You always have the option to withdraw from receiving this contact from us. Is that alright with you?
speaker: SPEAKER_00 - start: 31.0 - end: 36.0 - text:  Erm, sure, no problem now about the tea.
speaker: SPEAKER_01 - start: 36.0 - end: 44.0 - text:  Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box fro

In [99]:
transcriptions_by_speaker

{'SPEAKER_01': {'transcription': "  Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today?  Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded.  We may contact you in the future to offer further products and services.  You always have the option to withdraw from receiving this contact from us. Is that alright with you?  Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box from Shanghai to London would depend on a few factors,  such as the shipping method, delivery speed and any additional services you might require.  For an accurate quote, could you please provide me with more details? Are you looking for standard shipping or express?  Do you have any specific preferences?  Got it Bob, thank you for clarifying.  Standard shipping for a £50 box from Shanghai to London typically falls within £100 to £200 based on our current rates.  Please note that

In [90]:
dialogue_details

[{'speaker': 'SPEAKER_00',
  'transcription': ' Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today?'},
 {'speaker': 'SPEAKER_01',
  'transcription': " Hi Anna, it's Bob. I'm wondering about the cost to send a £50 box of tea from Shanghai to London."},
 {'speaker': 'SPEAKER_00',
  'transcription': " Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded."},
 {'speaker': 'SPEAKER_00',
  'transcription': ' We may contact you in the future to offer further products and services.'},
 {'speaker': 'SPEAKER_00',
  'transcription': ' You always have the option to withdraw from receiving this contact from us. Is that alright with you?'},
 {'speaker': 'SPEAKER_01',
  'transcription': ' Erm, sure, no problem now about the tea.'},
 {'speaker': 'SPEAKER_00',
  'transcription': ' Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box from Shanghai to London would d

In [95]:
def text_processing_summary(dialogue_details):
    conversation = ""
    for i in range(len(dialogue_details)):
        speaker_id = dialogue_details[i].get("speaker")
        transcription = dialogue_details[i].get("transcription")
        conversation = conversation + speaker_id + ":" + transcription + " "
    return conversation

In [96]:
text_processing_summary(dialogue_details)

"SPEAKER_00: Good afternoon, thank you for calling the ghost office. My name is Anna, how may I assist you today? SPEAKER_01: Hi Anna, it's Bob. I'm wondering about the cost to send a £50 box of tea from Shanghai to London. SPEAKER_00: Hi Bob, I'd be happy to help with that. Before we proceed, I need to let you know that this call is being recorded. SPEAKER_00: We may contact you in the future to offer further products and services. SPEAKER_00: You always have the option to withdraw from receiving this contact from us. Is that alright with you? SPEAKER_01: Erm, sure, no problem now about the tea. SPEAKER_00: Thank you for your understanding Bob. Now regarding your enquiry, sending a £50 box from Shanghai to London would depend on a few factors, SPEAKER_00: such as the shipping method, delivery speed and any additional services you might require. SPEAKER_00: For an accurate quote, could you please provide me with more details? Are you looking for standard shipping or express? SPEAKER_00